In [1]:
#Import all the dependencies
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, TaggedLineDocument
import gzip, os, glob
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('gutenberg')

In [2]:
from os import listdir
from os.path import isfile, join
LabeledSentence = gensim.models.doc2vec.LabeledSentence
import pandas as pd
import json

In [3]:
path = "state-of-the-union/transcripts.json"

with open(path) as data_file:
  data = data_file.read()
#   print(data) 
  data_content = json.loads(data)
  transcripts = [line['transcript'] for line in data_content]
  print(transcripts[0])

believe that patients with terminal conditions should have access to experimental treatments that could potentially save their lives.
People who are terminally ill should not have to go from country to country to seek a cure -- I want to give them a chance right here at home.  It is time for the Congress to give these wonderful Americans the "right to try."
One of my greatest priorities is to reduce the price of prescription drugs.  In many other countries, these drugs cost far less than what we pay in the United States.  That is why I have directed my Administration to make fixing the injustice of high drug prices one of our top priorities.  Prices will come down.
America has also finally turned the page on decades of unfair trade deals that sacrificed our prosperity and shipped away our companies, our jobs, and our Nation's wealth.
The era of economic surrender is over.
From now on, we expect trading relationships to be fair and to be reciprocal.
We will work to fix bad trade deals a

In [4]:
from nltk.corpus import stopwords
import re
import string

In [5]:
stop_words = set(stopwords.words('english'))

tagged_data = []
for i, _d in enumerate(transcripts):
    lowercase = _d.lower()
    remove_urls = re.sub(r"http\S+", "", lowercase)
    remove_punctuation = remove_urls.translate(str.maketrans('','',string.punctuation))
    tokenized = word_tokenize(remove_punctuation)
    words = [w for w in tokenized if not w in stop_words]
    tagged_data.append(TaggedDocument(words=words, tags=[str(i)]))

In [7]:
tagged_data

sentativesthe', 'sufficiency', 'revenues', 'established', 'objects', 'appropriated', 'leaves', 'doubt', 'residuary', 'provisions', 'commensurate', 'objects', 'public', 'faith', 'stands', 'pledged', 'allow', 'moreover', 'hope', 'favorite', 'policy', 'merely', 'secure', 'payment', 'interest', 'debt', 'funded', 'far', 'fast', 'growing', 'resources', 'country', 'permit', 'exonerate', 'principal', 'appropriation', 'made', 'western', 'land', 'explains', 'dispositions', 'subject', 'persuaded', 'sooner', 'valuable', 'fund', 'made', 'contribute', 'along', 'means', 'actual', 'reduction', 'public', 'debt', 'salutary', 'measure', 'every', 'public', 'interest', 'well', 'satisfactory', 'constituentsgentlemen', 'senate', 'house', 'representatives', 'pursuing', 'various', 'weighty', 'business', 'present', 'session', 'indulge', 'fullest', 'persuasion', 'consultation', 'equally', 'marked', 'wisdom', 'animated', 'love', 'country', 'whatever', 'belongs', 'duty', 'shall', 'cooperation', 'undiminished', 'ze

In [63]:
# tokenizer = RegexpTokenizer(r'/w+')
# stopword_set = set(stopwords.words('english'))

In [28]:
# https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
iteration 0
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 4

In [12]:
model = Doc2Vec.load("d2v.model")

In [24]:
model.wv.most_similar("patriot")

[('perpetuitywhen', 0.8139880895614624),
 ('moody', 0.7936184406280518),
 ('ulloa', 0.7933638691902161),
 ('coward', 0.7824451327323914),
 ('mankind', 0.7651658654212952),
 ('burr', 0.7540491223335266),
 ('unshakable', 0.7471182346343994),
 ('freedom—all', 0.745302677154541),
 ('beneficence', 0.7428082823753357),
 ('fitthe', 0.7425453066825867)]

In [26]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [28]:
doc_tags = list(model.docvecs.doctags.keys())
X = model[doc_tags]

In [31]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X)
df = pd.DataFrame(X_tsne, index=doc_tags, columns=['x', 'y'])

In [13]:
features_extracted = []
for i, review in enumerate(transcripts):
    features_extracted.append(model.docvecs[str(i)])

In [14]:
features_extracted

616139  , -2.3712604 ,  0.90280855,  2.517138  ,  0.46740505,
        -4.4376054 , -4.437304  ,  2.969196  , -0.83562005, -5.0835443 ,
        -4.927015  ,  7.167033  ,  5.043627  ,  2.1631894 ,  2.5042949 ,
         1.2331202 ,  2.0675197 ,  8.444792  ,  0.96488976, -1.6644232 ],
       dtype=float32),
 array([ 6.730798  , -2.9420183 ,  0.38327664,  3.4064264 ,  0.25522932,
        -4.318527  , -4.426588  ,  2.2039752 , -0.44356927, -4.365916  ,
        -5.3040023 ,  6.505506  ,  4.7078104 ,  2.6961608 ,  2.555988  ,
         0.7243362 ,  1.9843554 ,  8.401902  ,  1.3869754 , -0.86349   ],
       dtype=float32),
 array([ 9.422152  , -1.4157298 , -0.6058425 ,  2.052182  , -1.994156  ,
        -4.035746  , -3.9626226 ,  4.4627357 , -0.49113333, -4.107216  ,
        -5.345608  ,  6.6137733 ,  3.8687532 ,  1.7255117 ,  1.4287689 ,
         0.23918167,  2.7802365 ,  5.536721  ,  0.6593421 , -2.1914124 ],
       dtype=float32),
 array([ 9.75469   , -2.50314   , -0.5539391 ,  4.107713  ,  1.

In [15]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
scaled = ss.fit_transform(features_extracted)

In [16]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(scaled)

In [35]:
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=5, metric='cosine').fit(scaled)
_, closest = neighbors.kneighbors(kmeans.cluster_centers_)

In [36]:
for index, cluster_row in enumerate(closest):
    print('\n\nCluster number ' + str(index) + '\n\n')
    for col in cluster_row:
        print(features_extracted[col])
        print('\n\n')



Cluster number 0


[  0.43651295   4.492567     4.9231377   -1.8306363    4.914105
  10.763849     5.4608526   -1.3433133    4.9896145    7.67781
  -5.782557    -0.13995701  -1.1087203   -5.537945    -5.2405553
   6.4429264   -2.8201516  -11.19566     -0.8693175    5.2676187 ]



[  0.7787276    4.055593     4.9850755   -1.9606862    4.8718576
  10.546577     5.915137    -1.0424665    4.7958717    8.470183
  -4.3082066   -1.4644629   -0.85386366  -4.907125    -4.732545
   7.82673     -4.151798   -12.516549     0.90706044   5.6892643 ]



[ 1.7007109e+00  4.4497361e+00  6.1280117e+00 -7.9567426e-01
  5.1444764e+00  1.1334599e+01  5.8706388e+00  4.1351590e-01
  4.6619201e+00  8.0796471e+00 -5.1920161e+00 -1.4504055e+00
 -7.2643184e-03 -5.4557095e+00 -3.8766565e+00  7.4596925e+00
 -4.4451647e+00 -1.1641009e+01  2.1099645e-01  4.7739630e+00]



[  1.8893292    4.446615     5.138615    -2.0335073    3.8355625
  11.262195     5.5535483   -0.6406952    5.848924     8.018832
  -3.997432    -

In [37]:
import umap

embedding = umap.UMAP().fit_transform(scaled)

In [38]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(embedding)
embedding_scaled = scaler.transform(embedding)

In [39]:
embedding_scaled

array([[0.5751681 , 0.01005965],
       [0.812602  , 0.03641753],
       [0.826427  , 0.03763711],
       [0.92487323, 0.04174848],
       [0.9764662 , 0.05673469],
       [0.9580904 , 0.06117047],
       [1.        , 0.05319116],
       [0.99323094, 0.05514335],
       [0.9869726 , 0.05591649],
       [0.9867735 , 0.07070545],
       [0.91163105, 0.06228486],
       [0.91042525, 0.06911096],
       [0.8921868 , 0.06648156],
       [0.9172305 , 0.0679033 ],
       [0.91511804, 0.07528319],
       [0.89895725, 0.07570689],
       [0.90127116, 0.07484379],
       [0.93380785, 0.06545971],
       [0.9369468 , 0.04767026],
       [0.92767245, 0.05385   ],
       [0.94332874, 0.04968247],
       [0.9386639 , 0.05389816],
       [0.95741254, 0.03998524],
       [0.96563506, 0.04915641],
       [0.9633465 , 0.05106866],
       [0.9798003 , 0.06508063],
       [0.970838  , 0.06967847],
       [0.82871664, 0.0434377 ],
       [0.9470597 , 0.05597838],
       [0.8581205 , 0.05185936],
       [0.

In [41]:
with open(path) as data_file:
  data = data_file.read()
#   print(data) 
  data_content = json.loads(data)
  presidents = [line['president'] for line in data_content]
  print(president[100])

Herbert Hoover


In [45]:
lookup = []

with open(path) as data_file:
    data = data_file.read()
    data_content = json.loads(data)
    transcripts = [line['transcript'] for line in data_content]
    dates = [line['date'] for line in data_content]
    presidents = [line['president'] for line in data_content]
    for date, president, cluster_pos, transcript in zip(dates, presidents, embedding_scaled, transcripts):
        lookup.append({
            "Date": date,
            "President": president,
            "Cluster_Pos": cluster_pos.tolist(),
            "Speech": transcript
        })

In [46]:
with open('sotu_umap_position.json', 'w') as outfile:
    json.dump(lookup, outfile)